In [9]:
import os
os.environ['OPENAI_API_KEY'] = os.environ.get('OPENAI_API_KEY')

from langchain import OpenAI
model = OpenAI(model_name="gpt-3.5-turbo-instruct")



import pandas as pd
df = pd.DataFrame(columns=['flower_type','price','description','reason'])
flowers = ['玫瑰','百合','康乃馨']
prices = ['50','30','20']

from pydantic import BaseModel,Field
class FlowerDescription(BaseModel):
    flower_type: str = Field(description='鲜花的种类')
    price : int =  Field(description='鲜花的价格')    
    description: str = Field(description='鲜花的描述文案')
    reason: str = Field(description='为什么要这样的文案')

'''
from langchain.output_parsers import PydanticOutputParser
output_parser = PydanticOutputParser(pydantic_object=FlowerDescription)

format_instructions = output_parser.get_format_instructions()

print('输出格式',format_instructions)


from langchain import PromptTemplate
prompt_template= """您是一位专业的鲜花店文案撰写员，对于售价为{price}的{flower},您能提供一个吸引人的简短中文描述吗？
{format_instructions}
"""

prompt = PromptTemplate.from_template(prompt_template,partial_variables={"format_instructions":format_instructions})

print(prompt)'''

from langchain.output_parsers import PydanticOutputParser
output_parser = PydanticOutputParser(pydantic_object=FlowerDescription)
# 获取输出格式指示
format_instructions = output_parser.get_format_instructions()
# 打印提示
print("输出格式：",format_instructions)

from langchain import PromptTemplate
prompt_template = """您是一位专业的鲜花店文案撰写员。对于售价为 {price} 元的 {flower} ，您能提供一个吸引人的简短中文描述吗？{format_instructions}"""# 根据模板创建提示，同时在提示中加入输出解析器的说明
prompt = PromptTemplate.from_template(prompt_template, partial_variables={"format_instructions": format_instructions}) # 打印提示
print("提示：", prompt)

    

输出格式： The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"flower_type": {"description": "\u9c9c\u82b1\u7684\u79cd\u7c7b", "title": "Flower Type", "type": "string"}, "price": {"description": "\u9c9c\u82b1\u7684\u4ef7\u683c", "title": "Price", "type": "integer"}, "description": {"description": "\u9c9c\u82b1\u7684\u63cf\u8ff0\u6587\u6848", "title": "Description", "type": "string"}, "reason": {"description": "\u4e3a\u4ec0\u4e48\u8981\u8fd9\u6837\u7684\u6587\u6848", "title": "Reason", "type": "string"}}, "required": ["flower_type", "price", "description", "reason"]}
```
提示： input_va

In [10]:
for flower,price in zip(flowers,prices):
    input = prompt.format(flower=flower,price=price)
    #print('提示词',input)
    output = model.invoke(input)
    parsed_output =  output_parser.parse(output)
    parsed_output_dict = parsed_output.dict()
    df.loc[len(df)] = parsed_output.dict()

print('输出数据',df.to_dict(orient='records'))

输出数据 [{'flower_type': '玫瑰', 'price': 50, 'description': '这束美丽的玫瑰，花瓣鲜艳，香味浓郁，是爱情的象征。它能让您的心上人感受到您的浪漫和真挚的情感，是送礼的最佳选择。', 'reason': '玫瑰是最受人喜爱的花朵之一，它的美丽和芬芳能让人心情愉悦，同时也能表达出您的爱意和关怀。而且，这束玫瑰的价格实惠，是您表达爱意的最佳选择。'}, {'flower_type': '百合', 'price': 30, 'description': '这束美丽的百合，含蓄而又高雅，如同一位温婉的贵妇人，适合送给心仪的女生或者妈妈，传递出您对她们温柔的爱意。', 'reason': '百合象征纯洁和高贵，也代表着美好的祝愿，让您的爱意和祝福直达对方心中。'}, {'flower_type': '康乃馨', 'price': 20, 'description': '康乃馨是一种美丽的花朵，它象征着爱情和温柔。它的花朵绽放时呈现出淡粉色，非常柔美动人。无论是作为礼物还是装饰，都能给人带来温馨和浪漫的感觉。', 'reason': '康乃馨是一种常见的鲜花，价格实惠，适合送给亲朋好友或者自己欣赏。它也是表达爱意和感谢的最佳选择。'}]


In [19]:
from typing import List
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import OutputFixingParser

class Flower(BaseModel):
    name:str = Field(description='name of flower')
    colors:List[str] = Field(description='the colors of this flower')

flower_query = "Generate the characters for a random flower."
misformatted = "{'name':'康乃馨','colors':['粉红色','白色','红色','紫色','黄色']}"
misformatted = '{"name":"康乃馨","colors":["粉红色","白色","红色","紫色","黄色"]}'

parser = PydanticOutputParser(pydantic_object=Flower)

parser.parse(misformatted)
print(parser.parse(misformatted))

new_parser = OutputFixingParser.from_llm(parser=parser, llm=ChatOpenAI())
# 使用新的解析器解析不正确的输出
result = new_parser.parse(misformatted) # 错误被自动修正
#print(result) # 打印解析后的输出结果

name='康乃馨' colors=['粉红色', '白色', '红色', '紫色', '黄色']


In [32]:
template = """Based on the user question,provide an Action and Iuput for what step should be taken.
{format_instructions}
Question: {query}
Response: """

class Action(BaseModel):
    action:str = Field(description='action to take')
    action_input:str = Field(description='inuout to the action')

parser = PydanticOutputParser(pydantic_object=Action)

prompt = PromptTemplate(
    template = "Answer the user query.\n {format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables= {"format_instructions":parser.get_format_instructions()},
)
'''
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)'''
prompt_value = prompt.format_prompt(query="what are the colors of orchild?")
bad_response = '{"action":"search"}'


#parser.parse(bad_response)

fix_parser = OutputFixingParser.from_llm(parser=parser, llm=ChatOpenAI())
#fix_parser.parse(bad_response)



from langchain.output_parsers import RetryWithErrorOutputParser
from langchain.llms import OpenAI
retry_parser = RetryWithErrorOutputParser.from_llm( parser=parser, llm=OpenAI(temperature=0))
parse_result = retry_parser.parse_with_prompt(bad_response, prompt_value)
print('RetryWithErrorOutputParser的parse结果:',parse_result)


RetryWithErrorOutputParser的parse结果: action='search' action_input='colors of orchid'
